In [1]:
using TuringModels
using Turing

Turing.setadbackend(:reverse_diff);
Turing.turnprogress(false);

d = CSV.read(rel_path("..", "data", "Kline.csv"), delim=';');
size(d) # Should be 10x5

┌ Info: [Turing]: global PROGRESS is set as false
└ @ Turing /Users/rob/.julia/packages/Turing/UXxKz/src/Turing.jl:24


(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map(x -> log(x), d[:population]);

New col contact_high, set binary values 1/0 if high/low contact

In [3]:
d[:contact_high] = map(x -> ifelse(x=="high", 1, 0), d[:contact]);

This is supposed to be a "bad" model since we take non-centered data for the
predictor log_pop

In [4]:
@model m10_10stan(total_tools, log_pop, contact_high) = begin
    α ~ Normal(0, 100)
    βp ~ Normal(0, 1)
    βc ~ Normal(0, 1)
    βpc ~ Normal(0, 1)

    for i ∈ 1:length(total_tools)
        λ = exp(α + βp*log_pop[i] + βc*contact_high[i] +
            βpc*contact_high[i]*log_pop[i])
        total_tools[i] ~ Poisson(λ)
    end
end;

posterior = sample(m10_10stan(d[:total_tools], d[:log_pop],
    d[:contact_high]), Turing.NUTS(2000, 1000, 0.95));
# Fix the inclusion of adaptation samples
posterior2 = MCMCChain.Chains(posterior.value[1001:2000,:,:], names=posterior.names);

┌ Info: [Turing] looking for good initial eps...
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/support/hmc_core.jl:240
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:168
┌ Warning: grad = [NaN, NaN, NaN, NaN]
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:169
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:168
┌ Warning: grad = [NaN, NaN, NaN, NaN]
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:169
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:168
┌ Warning: grad = [NaN, NaN, NaN, NaN]
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:169
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/c

Rethinking result

In [5]:
m_10_10t_result = "
     mean   sd  5.5% 94.5% n_eff Rhat
 a    0.94 0.37  0.36  1.53  3379    1
 bp   0.26 0.04  0.21  0.32  3347    1
 bc  -0.08 0.84 -1.41  1.23  2950    1
 bpc  0.04 0.09 -0.10  0.19  2907    1
";

Describe the draws

In [6]:
describe(posterior2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1
Samples per chain = 1000

Empirical Posterior Estimates:
              Mean           SD        Naive SE               MCSE                ESS    
       α   0.8996068994  0.396412125 0.0125356521 0.0586794766824411326289201   45.637453
  lf_num   0.0000000000  0.000000000 0.0000000000 0.0000000000000000000000000         NaN
     βpc   0.0497646397  0.083275101 0.0026333899 0.0189400158917350548704039   19.331683
      βc  -0.1462900470  0.752607228 0.0237995303 0.1706291098780035320192638   19.454968
 elapsed   0.0289675535  0.016577288 0.0005242199 0.0005050476037407463118442 1000.000000
 epsilon   0.0044252662  0.000000000 0.0000000000 0.0000000000000000002891206    0.000000
      βp   0.2675817601  0.038030768 0.0012026385 0.0055090118616234647064278   47.656569
      lp -45.7827885824  1.473393073 0.0465927800 0.0969509819896755553614298  230.957900
eval_num 139.8100000000 71.726652038 2.2681958938 2.3155172208385761400961655  9

End of m_10_10t.jl

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*